# ■ 출력층 함수

 " 출력층의 함수는 그동안 흘러왔던 확률들의 숫자를 취합해서 결론을 내줘야하는 함수 "  

신경망으로 구현하고자 하는 문제

    1. 회귀 ?  항등함수 ---> 입력값을 받아 그대로 출력하는 함수

    예: R 을 활용한 머신러닝때 콘크리트 강도 예측하는 머신러닝 모델을 생성
        이때는 분류를 한게 아니라 회귀를 통한 수치를 예측(콘크리트 강도)

    독립변수 : 콘크리트 재료: 자갈 200kg, 시멘트 20 포대 …..
    종속변수 : 콘크리트 강도

    2. 분류 ?  소프트맥스 함수 ---> 책 p91의 식 3.10
	    		: 입력값을 받아서 확률벡터로 출력하는 함수

	정상 폐사진 vs 폐결절 사진, 수화동작을 글로 출력하는 신경망

# ■ 출력층 함수인 소프트맥스 함수 생성

$$y_k = \frac{exp(a_k)}{\sum_{i=1}^n exp(a_k)}$$

위의 식을 파이썬으로 그대로 만들면 에러가 나서 구현이 안됩니다.  
왜냐하면 지수함수는 아주 큰 값을 쉽게 출력하기 때문에   
컴퓨터는 큰 값이 출력이 되면 overflow 가 출력되면서 에러가 납니다.  
위의 수학식을 컴퓨터로 구현할 수 있게 하려면 아래와 같이 전해해줘야 합니다.  

$$y_k = \frac{exp(a_k)}{\sum_{i=1}^n exp(a_i)} = \frac{C exp(a_k)}{C  \sum_{i=1}^n exp(a_i)}=\frac{exp(a_k+\log C)}{\sum_{i=1}^n exp(a_i+\log C)}=\frac{exp(a_k+\hat {C})}{\sum_{i=1}^n exp(a_i+ \hat{C})}$$

소프트맥스 함수의 자연상수의 지수함수는 아주 큰 값을 출력합니다.  
자연상수 e 의 10승은 20000 이 넘고 e 의 100승 숫자 40개가 넘고  
e 의 1000승은 무한대를 뜻하는 inf 가 출력됩니다. 그래서 계산을 할 수 없습니다.

## 예시

In [1]:
import numpy as np
print( np.exp(10) )
print( np.exp(100) )
print( np.exp(1000) )

22026.465794806718
2.6881171418161356e+43
inf


<ipython-input-1-53d45450e635>:4: RuntimeWarning: overflow encountered in exp
  print( np.exp(1000) )


## 예시. 이를 해결하기 위해서 입력 신호 값들의 최대값에서 각각의 요소의 값을 빼줍니다.

In [2]:
import numpy as np
a = np.array([1010, 1000, 990])
print( np.exp(a) )

[inf inf inf]


<ipython-input-2-1a074ef2ffcb>:3: RuntimeWarning: overflow encountered in exp
  print( np.exp(a) )


In [3]:
import numpy as np
a = np.array([1010, 1000, 990])
C = np.max(a)
minus = a - C
print( np.exp(minus) )

[1.00000000e+00 4.53999298e-05 2.06115362e-09]


## 위의 코드를 이용해서 소프트 맥스 함수 구현하기 (아래 식의 분자 구현)

$$y_k = \frac{exp(a_k)}{\sum_{i=1}^n exp(a_i)} = \frac{C exp(a_k)}{C  \sum_{i=1}^n exp(a_i)}=\frac{exp(a_k+\log C)}{\sum_{i=1}^n exp(a_i+\log C)}=\frac{exp(a_k+\hat {C})}{\sum_{i=1}^n exp(a_i+ \hat{C})}$$

In [4]:
a = np.array([1010, 1000, 990])

def softmax(a):
    C = np.max(a)
    minus = a - C
    np_exp = np.exp(minus)
    return np_exp

print(softmax(a))

[1.00000000e+00 4.53999298e-05 2.06115362e-09]


## 분모까지 포함해서 구현하면 ?

In [5]:
import numpy as np

a = np.array([1010, 1000, 990])

def softmax(a):
    C = np.max(a)
    minus = a - C
    exp_a = np.exp(minus)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

print(softmax(a))

[9.99954600e-01 4.53978686e-05 2.06106005e-09]


## 문제46. 위의 결과 리스트의 요소를 다 더하면 숫자 1인지 출력하시오

In [6]:
import numpy as np

a = np.array([1010, 1000, 990])

def softmax(a):
    C = np.max(a)
    minus = a - C
    exp_a = np.exp(minus)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

print( np.sum(softmax(a)))

1.0


## 문제47. 위의 결과 리스트 요소 3개중에 어떤게 가장 큰 값인지 인덱스 번호로 출력하시오

In [7]:
import numpy as np

a = np.array([1010, 1000, 990])

def softmax(a):
    C = np.max(a)
    minus = a - C
    exp_a = np.exp(minus)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

print( np.argmax(softmax(a)) )   

0


※ np.argmax : numpy 리스트 요소중 큰 값의 인덱스 번호를 출력하는 함수

## 문제48. 방금 만든 소프트맥스 함수를 문제45번에 만든 3층 신경망 맨 끝 출력층에 k 값을 입력받도록 구현하시오! 출력은 k_hat 으로 하세요

In [8]:
import numpy as np

# 신경망 함수들
def sigmoid(x):
    return 1 / (1 + np.exp(-x) )

def softmax(a):
    C = np.max(a)
    minus = a - C
    exp_a = np.exp(minus)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

# 0층
x = np.array( [1,2] )

# 1층
w1 = np.array([[1,3,5], [2,4,6]])
y = np.dot(x,w1)
y_hat = sigmoid(y)

# 2층
w2 = np.array([[3,4], [5,6], [7,8]])
z = np.dot(y_hat, w2)
z_hat = sigmoid(z)

# 3층
w3 = np.array([[4,5], [6,7]])
k = np.dot(z_hat, w3)
k_hat = softmax(k)
print(k_hat)

[0.11920296 0.88079704]


## 문제49. 위의 3층 신경망 코드에서 w1, w2, w3 가중치를 하나로 모아서 심플한 코드로 작성하시오 !

 " 딕셔너리를 활용하면 됩니다. "  

** 파이썬의 자료형 5가지

    1. 문자형  
    2. 숫자형  
    3. 리스트형  
    4. 딕셔너리형 : 키와 값으로 구성되어 있는 자료구조
    5. 튜플

In [2]:
import numpy as np

def init_network():
    network = {}  # 비어있는 딕셔너리 생성
    network['W1'] = np.array([[1,3,5], [2,4,6]])
    network['W2'] = np.array([[3,4], [5,6], [7,8]])
    network['W3'] = np.array([[4,5], [6,7]]) 
    return network

# 가중치 값을 불러온다.
network = init_network()
w1, w2, w3 = network['W1'], network['W2'], network['W3']

# 신경망 함수들
def sigmoid(x):
    return 1 / (1 + np.exp(-x) )

def softmax(a):
    C = np.max(a)
    minus = a - C
    exp_a = np.exp(minus)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

# 0층
x = np.array( [1,2] )

# 1층
y = np.dot(x,w1)
y_hat = sigmoid(y)

# 2층
z = np.dot(y_hat, w2)
z_hat = sigmoid(z)

# 3층
k = np.dot(z_hat, w3)
k_hat = softmax(k)
print(k_hat)

[0.11920296 0.88079704]


## 문제50. 위의 sigmoid, softmax, init_network 함수코드를 common.py 라는 이름으로 메모장에 저장하고 파이썬의 워킹디렉토리에 저장하시오 !

In [10]:
%pwd   # 워킹 디렉토리 확인

'c:\\data\\jupyter\\deep_learning\\2021.03.08'

<common.py>

In [ ]:
import numpy as np

def init_network():
    network = {}  # 비어있는 딕셔너리 생성
    network['W1'] = np.array([[1,3,5], [2,4,6]])
    network['W2'] = np.array([[3,4], [5,6], [7,8]])
    network['W3'] = np.array([[4,5], [6,7]]) 
    return network

# 신경망 함수들
def sigmoid(x):
    return 1 / (1 + np.exp(-x) )

def softmax(a):
    C = np.max(a)
    minus = a - C
    exp_a = np.exp(minus)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

In [1]:
import numpy as np
from common import init_network, sigmoid, softmax

# 가중치 값을 불러온다.
network = init_network()
w1, w2, w3 = network['W1'], network['W2'], network['W3']

# 0층
x = np.array( [1,2] )

# 1층
y = np.dot(x,w1)
y_hat = sigmoid(y)

# 2층
z = np.dot(y_hat, w2)
z_hat = sigmoid(z)

# 3층
k = np.dot(z_hat, w3)
k_hat = softmax(k)
print(k_hat)

[0.11920296 0.88079704]
